In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
pip install scikit-surprise


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp39-cp39-linux_x86_64.whl size=3195810 sha256=3149e9c9118d25ec32a61c90db0c209de9e91facc9aeb2065e280f7ba2c50ea7
  Stored in directory: /root/.cache/pip/wheels/c6/3a/46/9b17b3512bdf283c6cb84f59929cdd5199d4e754d596d22784
Successfully built scikit-surprise


In [3]:
pip install implicit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.6/18.6 MB 70.5 MB/s eta 0:00:00


In [4]:
from surprise import Dataset, Reader
from surprise import SVD, SVDpp, BaselineOnly

from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate

In [5]:
from scipy.sparse import csr_matrix

from implicit.als import AlternatingLeastSquares
from implicit.lmf import LogisticMatrixFactorization

In [6]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 30)

In [7]:
random_state_value = 42

id_column_name = 'common_id_2'
rating_column_name = 'rating'

**Загружаем датасет**

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
data_dir = '/content/drive/MyDrive/хакатон_работа_ру'

In [11]:
train_df = pd.read_parquet(f"{data_dir}/Processed_dataset1.parquet")
train_df

,common_id,event_date,event_timestamp,action_time,vacancy_id_,cookie_id,user_id,event_type,common_id_2
15,2,2022-09-05,1662374899,0.0,192850,0000d7508334414ca792c5ff66eb8c14,None,preview_click_vacancy,0000d7508334414ca792c5ff66eb8c14
16,2,2022-09-05,1662374899,498.0,192850,0000d7508334414ca792c5ff66eb8c14,None,show_vacancy,0000d7508334414ca792c5ff66eb8c14
17,2,2022-09-05,1662375397,1.0,230581,0000d7508334414ca792c5ff66eb8c14,None,preview_click_vacancy,0000d7508334414ca792c5ff66eb8c14
18,2,2022-09-05,1662375398,205.0,230581,0000d7508334414ca792c5ff66eb8c14,None,show_vacancy,0000d7508334414ca792c5ff66eb8c14
19,2,2022-09-05,1662375603,1.0,250327,0000d7508334414ca792c5ff66eb8c14,None,preview_click_vacancy,0000d7508334414ca792c5ff66eb8c14
...,...,...,...,...,...,...,...,...,...
12292571,367640,2022-09-26,1664169217,1.0,197828,ffffd6d4f2134dd6aa80ff71e942b508,82a5e80f91b144f596496d6d831b49d3,preview_click_vacancy,82a5e80f91b144f596496d6d831b49d3
12292572,367640,2022-09-26,1664169218,90.0,197828,ffffd6d4f2134dd6aa80ff71e942b508,82a5e80f91b144f596496d6d831b49d3,show_vacancy,82a5e80f91b144f596496d6d831b49d3
12292573,367640,2022-09-26,1664169308,210911.0,197828,ffffd6d4f2134dd6aa80ff71e942b508,82a5e80f91b144f596496d6d831b49d3,click_contacts,82a5e80f91b144f596496d6d831b49d3
12292574,367640,2022-09-28,1664380219,132.0,233452,ffffd6d4f2134dd6aa80ff71e942b508,82a5e80f91b144f596496d6d831b49d3,click_contacts,82a5e80f91b144f596496d6d831b49d3


In [12]:
test_df = pd.read_parquet(f"{data_dir}/test_public_mfti.parquet")
test_df  

,cookie_id,vacancy_id_
0,000cd76cd33f43d4a1ac1d16d10f8bf7,"[222177, 222173, 222163, 238874, 238878, 228125, 225604, 235544, 119314]"
1,0034bc7f404341ba8412665453e7825a,"[102794, 137587, 257319, 237756, 240744, 113482, 240245, 221376, 242642, 235561, 191512, 222762, 231446, 109269, 205578]"
2,00a6c5a64a274c55a836402bdeb3b2c4,"[254292, 164602, 116438, 228634, 218819, 240655, 240659, 237341, 176563, 255299, 245637, 109275, 111505, 212953, 217549, 207108, 200475, 211355, 211449, 108956, 111071, 253239, 109649, 221735, 222030, 222174, 231207, 231022, 234380, 106944, 203948, 175325, 193534, 197505, 234769, 238933, 225709, 184466, 242642, 203015]"
3,015937a125b14e74bdff1cddc49f9172,"[246685, 138123, 115420, 210628, 212325, 235196]"
4,01de50c280794cec8804f16f45f847b7,"[219070, 251469, 166899, 212703, 214561]"
...,...,...
767,fdbcda17f22f406486837059e76c7fed,"[207851, 254989, 213344, 214180, 222146]"
768,fe6193ab26494ace9be5aae36e507618,"[115352, 230546, 225527, 120188, 109360, 232128, 237590, 259570, 244126, 229677, 135447]"
769,fe95b2826ee1452b81201ed3f4c3294d,"[240362, 114852, 253946, 251081, 127546, 244688]"
770,ff1aef256a49481698bb2e938510ff36,"[231194, 236363, 220747, 244688, 100094, 240521, 179568, 184858, 100103]"


In [13]:
nonrel_act_list = ['show_vacancy',
                'preview_click_vacancy',
                'click_favorite',
                'preview_click_favorite']

rel_act_list = ['click_response',
                'preview_click_response',
                'click_contacts',
                'preview_click_contacts',
                'click_phone',
                'preview_click_phone']

In [14]:
def get_top_n_vacancies(rating_act_list, pos_vacancies_start=0, pos_vacancies_end=100):
    show_act = train_df[train_df['event_type'].isin(rating_act_list)].groupby('vacancy_id_')['event_type'].count()
    top_n_with_event_quant = show_act.sort_values(ascending=False).iloc[pos_vacancies_start:pos_vacancies_end].to_frame().reset_index()
    return top_n_with_event_quant['vacancy_id_'].unique()

In [ ]:
# ТОП-80 для рейтинга по просмотрам

vac_start = 0
vac_end = 80
rating_act = ['show_vacancy']

top80vac = get_top_n_vacancies(rating_act,
                               pos_vacancies_start=vac_start,
                               pos_vacancies_end=vac_end)
top80vac

In [ ]:
# ТОП-200 для рейтинга по просмотрам

vac_start = 0
vac_end = 200
rating_act = ['show_vacancy']

top200vac = get_top_n_vacancies(rating_act,
                                pos_vacancies_start=vac_start,
                                pos_vacancies_end=vac_end)
top200vac

In [ ]:
# ТОП-1000 для рейтинга по просмотрам

vac_start = 0
vac_end = 1000
rating_act = ['show_vacancy']

top1000vac = get_top_n_vacancies(rating_act,
                                pos_vacancies_start=vac_start,
                                pos_vacancies_end=vac_end)
top1000vac

In [ ]:
# Топ вакансий с 10 до 95

vac_start = 10
vac_end = 95
rating_act = ['show_vacancy']

top95vac_from10 = get_top_n_vacancies(rating_act,
                                                      pos_vacancies_start=vac_start,
                                                      pos_vacancies_end=vac_end)
top95vac_from10

array([250327, 111867, 110421, 108242, 242642, 217683, 258441, 158242,
       162187, 174953, 176141, 207108, 247535, 110792, 237341, 113305,
       240744, 149024, 244077, 113482, 180382, 114583, 182100, 113707,
       105907, 210628, 227708, 110793, 247276, 181745, 164481, 193331,
       246509, 243868, 190030, 153245, 138634, 169194, 115924, 126251,
       164588, 154411, 155539, 120252, 230707, 127352, 214513, 182084,
       249571, 207156, 168935, 239021, 184440, 106944, 187360, 206350,
       117525, 111592, 150283, 253678, 112506, 117532, 220718, 151616,
       109079, 111837, 136266, 164246, 182439, 111890, 257631, 143721,
       248852, 171332, 128183, 140917, 111941, 212141, 102794, 212325,
       229689, 258378, 106293, 176171, 129787])

In [16]:
# Создаем колонку с типом действия для рейтинга
# 1 - целевое действие, 0 - остальные действия

def check_is_purpose(event):
    if event in rel_act_list:
        return 1
    else:
        return 0

train_df[rating_column_name] = train_df['event_type'].apply(check_is_purpose)
train_df

,common_id,event_date,event_timestamp,action_time,vacancy_id_,cookie_id,user_id,event_type,common_id_2,rating
15,2,2022-09-05,1662374899,0.0,192850,0000d7508334414ca792c5ff66eb8c14,None,preview_click_vacancy,0000d7508334414ca792c5ff66eb8c14,0
16,2,2022-09-05,1662374899,498.0,192850,0000d7508334414ca792c5ff66eb8c14,None,show_vacancy,0000d7508334414ca792c5ff66eb8c14,0
17,2,2022-09-05,1662375397,1.0,230581,0000d7508334414ca792c5ff66eb8c14,None,preview_click_vacancy,0000d7508334414ca792c5ff66eb8c14,0
18,2,2022-09-05,1662375398,205.0,230581,0000d7508334414ca792c5ff66eb8c14,None,show_vacancy,0000d7508334414ca792c5ff66eb8c14,0
19,2,2022-09-05,1662375603,1.0,250327,0000d7508334414ca792c5ff66eb8c14,None,preview_click_vacancy,0000d7508334414ca792c5ff66eb8c14,0
...,...,...,...,...,...,...,...,...,...,...
12292571,367640,2022-09-26,1664169217,1.0,197828,ffffd6d4f2134dd6aa80ff71e942b508,82a5e80f91b144f596496d6d831b49d3,preview_click_vacancy,82a5e80f91b144f596496d6d831b49d3,0
12292572,367640,2022-09-26,1664169218,90.0,197828,ffffd6d4f2134dd6aa80ff71e942b508,82a5e80f91b144f596496d6d831b49d3,show_vacancy,82a5e80f91b144f596496d6d831b49d3,0
12292573,367640,2022-09-26,1664169308,210911.0,197828,ffffd6d4f2134dd6aa80ff71e942b508,82a5e80f91b144f596496d6d831b49d3,click_contacts,82a5e80f91b144f596496d6d831b49d3,1
12292574,367640,2022-09-28,1664380219,132.0,233452,ffffd6d4f2134dd6aa80ff71e942b508,82a5e80f91b144f596496d6d831b49d3,click_contacts,82a5e80f91b144f596496d6d831b49d3,1


In [30]:
# Выбрать только те cookie_id, у которых количество вакансий больше 8 (1000 cлуйчаных) 
 
import random
random.seed(13)
# Выбрать cookie_id с количеством vacancy_id_ > 8 и рейтингом == 1
cookie_ids_to_exclude = train_df.groupby('cookie_id').filter(lambda x: (x['vacancy_id_'].nunique() > 8) and (x[x['rating'] == 1]['vacancy_id_'].nunique() >= 5))['cookie_id'].unique()

# Выбрать случайные 1000 уникальных cookie_id
cookie_ids_to_exclude = random.sample(list(cookie_ids_to_exclude), 1000)

# Выбрать только те cookie_id, которые нужно исключить
train_5_exclude = train_df[train_df['cookie_id'].isin(cookie_ids_to_exclude)]

# Получить последние 5 vacancy_id_ с рейтингом == 1 для каждого cookie_id исключаемых из train_5
test_5 = train_5_exclude.groupby('cookie_id').apply(lambda x: x[x['rating'] == 1].tail(5)).reset_index(drop=True)

merged = train_df.merge(test_5[['cookie_id', 'vacancy_id_']], how='left', on=['cookie_id', 'vacancy_id_'], indicator=True)
train_5 = merged[merged['_merge'] == 'left_only'].drop('_merge', axis=1)

In [33]:
train_5

,common_id,event_date,event_timestamp,action_time,vacancy_id_,cookie_id,user_id,event_type,common_id_2,rating
0,2,2022-09-05,1662374899,0.0,192850,0000d7508334414ca792c5ff66eb8c14,None,preview_click_vacancy,0000d7508334414ca792c5ff66eb8c14,0
1,2,2022-09-05,1662374899,498.0,192850,0000d7508334414ca792c5ff66eb8c14,None,show_vacancy,0000d7508334414ca792c5ff66eb8c14,0
2,2,2022-09-05,1662375397,1.0,230581,0000d7508334414ca792c5ff66eb8c14,None,preview_click_vacancy,0000d7508334414ca792c5ff66eb8c14,0
3,2,2022-09-05,1662375398,205.0,230581,0000d7508334414ca792c5ff66eb8c14,None,show_vacancy,0000d7508334414ca792c5ff66eb8c14,0
4,2,2022-09-05,1662375603,1.0,250327,0000d7508334414ca792c5ff66eb8c14,None,preview_click_vacancy,0000d7508334414ca792c5ff66eb8c14,0
...,...,...,...,...,...,...,...,...,...,...
12299785,367640,2022-09-26,1664169217,1.0,197828,ffffd6d4f2134dd6aa80ff71e942b508,82a5e80f91b144f596496d6d831b49d3,preview_click_vacancy,82a5e80f91b144f596496d6d831b49d3,0
12299786,367640,2022-09-26,1664169218,90.0,197828,ffffd6d4f2134dd6aa80ff71e942b508,82a5e80f91b144f596496d6d831b49d3,show_vacancy,82a5e80f91b144f596496d6d831b49d3,0
12299787,367640,2022-09-26,1664169308,210911.0,197828,ffffd6d4f2134dd6aa80ff71e942b508,82a5e80f91b144f596496d6d831b49d3,click_contacts,82a5e80f91b144f596496d6d831b49d3,1
12299788,367640,2022-09-28,1664380219,132.0,233452,ffffd6d4f2134dd6aa80ff71e942b508,82a5e80f91b144f596496d6d831b49d3,click_contacts,82a5e80f91b144f596496d6d831b49d3,1


In [31]:
test_5 = test_5.rename(columns={'removed_vacancies':'vacancy_id_'})

In [44]:
train_df.query("cookie_id == '0010106496d14232aa48897c62182503'")

,common_id,event_date,event_timestamp,action_time,vacancy_id_,cookie_id,user_id,event_type,common_id_2,rating
2418,82,2022-08-04,1659613481,3.0,248978,0010106496d14232aa48897c62182503,c844599efde04ad18ae29c72d13f87ce,click_contacts,c844599efde04ad18ae29c72d13f87ce,1
2419,82,2022-08-04,1659613484,38.0,248978,0010106496d14232aa48897c62182503,c844599efde04ad18ae29c72d13f87ce,click_phone,c844599efde04ad18ae29c72d13f87ce,1
2420,82,2022-08-04,1659613522,13.0,248978,0010106496d14232aa48897c62182503,c844599efde04ad18ae29c72d13f87ce,click_response,c844599efde04ad18ae29c72d13f87ce,1
2421,82,2022-08-04,1659613535,1.0,248978,0010106496d14232aa48897c62182503,c844599efde04ad18ae29c72d13f87ce,click_contacts,c844599efde04ad18ae29c72d13f87ce,1
2422,82,2022-08-04,1659613536,421.0,248978,0010106496d14232aa48897c62182503,c844599efde04ad18ae29c72d13f87ce,click_phone,c844599efde04ad18ae29c72d13f87ce,1
...,...,...,...,...,...,...,...,...,...,...
2466,82,2022-08-21,1661043413,6.0,250942,0010106496d14232aa48897c62182503,c844599efde04ad18ae29c72d13f87ce,preview_click_vacancy,c844599efde04ad18ae29c72d13f87ce,0
2467,82,2022-08-21,1661043419,16.0,250942,0010106496d14232aa48897c62182503,c844599efde04ad18ae29c72d13f87ce,show_vacancy,c844599efde04ad18ae29c72d13f87ce,0
2468,82,2022-08-21,1661043435,1.0,250942,0010106496d14232aa48897c62182503,c844599efde04ad18ae29c72d13f87ce,click_contacts,c844599efde04ad18ae29c72d13f87ce,1
2469,82,2022-08-21,1661043436,17.0,250942,0010106496d14232aa48897c62182503,c844599efde04ad18ae29c72d13f87ce,click_phone,c844599efde04ad18ae29c72d13f87ce,1


In [32]:
test_6 = test_5.groupby(['cookie_id'], as_index=False).agg({'vacancy_id_': list})
test_6

,cookie_id,vacancy_id_
0,0010106496d14232aa48897c62182503,"[131696, 162614, 184607, 250942, 250942]"
1,00432521aaff41b0b7e9df18a32ad440,"[227879, 122139, 122139, 155280, 149024]"
2,007a06bac390472c8d2b5c9296756d21,"[179082, 174953, 182870, 114583, 217606]"
3,0083e7acaae74f8596f3e85858a041db,"[168020, 125652, 248114, 171909, 156017]"
4,008f7b87a31b4f6e8b3958ee61a3224e,"[135422, 206549, 206549, 234115, 115977]"
...,...,...
995,fd23ed3759034783b497b9a99bcedb73,"[252559, 106268, 180086, 180086, 180086]"
996,fd6adcf3a10d4ad4bf1c74a03c1551ee,"[187597, 187597, 111101, 111101, 111101]"
997,fe8680d14913420fbadfbee3962a06f4,"[181555, 182240, 182821, 177818, 186323]"
998,ff584e73f5db469a801f99546bd8ed40,"[102471, 245040, 172964, 204687, 163764]"


In [34]:
# Готовим датафрейм для обучения модели:

train_matrix = train_5.groupby([id_column_name, 'vacancy_id_'], as_index=False)[rating_column_name].sum()
train_matrix

,common_id_2,vacancy_id_,rating
0,000089d26cdd49d68839c68bc10f2cf2,177380,0
1,000089d26cdd49d68839c68bc10f2cf2,181353,0
2,000089d26cdd49d68839c68bc10f2cf2,195890,0
3,000089d26cdd49d68839c68bc10f2cf2,205562,0
4,000089d26cdd49d68839c68bc10f2cf2,211064,0
...,...,...,...
4703352,ffffdb17f45b4032b386d691d52e6c00,107125,1
4703353,ffffdb17f45b4032b386d691d52e6c00,107893,1
4703354,ffffdb17f45b4032b386d691d52e6c00,150843,1
4703355,ffffdb17f45b4032b386d691d52e6c00,160164,1


In [35]:
# Функция для создания рейтинга на основе порогового значения для целевых действий

def create_rating(input_rating, thresholds):
    rating = input_rating
    if rating >= max(rating_thresholds):
        rating = thresholds[max(rating_thresholds)]
    else:
        rating = thresholds[rating_thresholds[rating]]
    return rating

In [36]:
# Создаем рейтинг
# Принципы построения
# 1. rating >= 1 - rating = 1
# 2. rating = 0 - оставляем без изменения

rating_thresholds = {0: 0, 1: 1}

train_matrix_0_1 = train_matrix.copy()
train_matrix_0_1[rating_column_name] = train_matrix_0_1[rating_column_name].apply(create_rating, thresholds = rating_thresholds)
train_matrix_0_1
train_matrix_0_1[rating_column_name].unique()

array([0, 1])

In [37]:
# Готовим данные для обучения модели

reader = Reader(rating_scale=(0, 1))
train_data_0_1 = Dataset.load_from_df(train_5[[id_column_name, 'vacancy_id_', rating_column_name]], reader)

In [38]:
# Обучаем модель для датасета с рейтингом 0 или 1

svd_20epochs_0_1 = SVD(verbose=True, n_epochs=20)
trainset_0_1 = train_data_0_1.build_full_trainset()
svd_20epochs_0_1.fit(trainset_0_1)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19


In [39]:
col_names ={
    'id_col_name': id_column_name,
    'vac_col_name': 'vacancy_id_',
    'rating_col_name': rating_column_name
}

In [40]:
def transform_into_sparse_matrix(input_df, col_names):
    input_df_fixed = input_df.copy()
    input_df_fixed[col_names['id_col_name']] = input_df_fixed[col_names['id_col_name']].astype('category')
    input_df_fixed[col_names['vac_col_name']] = input_df_fixed[col_names['vac_col_name']].astype('category')
    input_df_fixed[col_names['rating_col_name']] = input_df_fixed[col_names['rating_col_name']].astype('uint8')
    user_item_matrix = csr_matrix((input_df_fixed[col_names['rating_col_name']], 
                                (input_df_fixed[col_names['id_col_name']].cat.codes, 
                                 input_df_fixed[col_names['vac_col_name']].cat.codes)))
    id_cat_codes = dict(zip(input_df_fixed[col_names['id_col_name']], input_df_fixed[col_names['id_col_name']].cat.codes))
    vac_cat_codes = dict(zip(input_df_fixed[col_names['vac_col_name']], input_df_fixed[col_names['vac_col_name']].cat.codes))
    cat_vac_codes = dict(zip(input_df_fixed[col_names['vac_col_name']].cat.codes, input_df_fixed[col_names['vac_col_name']]))
    return (user_item_matrix, id_cat_codes, vac_cat_codes, cat_vac_codes)


In [41]:
Impl_0_1_set = transform_into_sparse_matrix(train_matrix_0_1, col_names)

In [42]:
Impl_ALS_0_1 = AlternatingLeastSquares(factors=100,
                                       regularization=0.05,
                                       calculate_training_loss=True,
                                       random_state=random_state_value) # , alpha=3.0
Impl_ALS_0_1.fit(Impl_0_1_set[0])

  0%|          | 0/15 [00:00<?, ?it/s]

In [45]:
# Вспомогательные функции

def get_predictions(cookie_id, model_info, initial_set, n_predict = 5):     
    model = model_info[0]
    model_type = model_info[1]  

    top_n_vacancies = []
    common_ids = common_for_cookie(cookie_id)
    for com_id, n in common_ids.items():
        if n != 0:
            # Получим все вакансии с которыми взаимодействовал cookie_id:
            interacted_items = train_matrix[train_matrix[id_column_name] == com_id]['vacancy_id_'].unique()
            
            # Удалим из этого перечня вакансии с которыми контактировал юзер:
            items_to_predict = np.setdiff1d(initial_set, interacted_items)
            
            # получим топ-n вакансий для данного юзера:
            if model_type == 'surprise':
                new_top_n_vac = get_predictions_surprise(com_id, model, items_to_predict, n)  
            
            if model_type == 'implicit':
                filtered_set = train_matrix[~train_matrix['vacancy_id_'].isin(items_to_predict)]['vacancy_id_'].unique()
                new_top_n_vac = get_predictions_implicit(com_id, model, filtered_set, model_info[2], n)
            top_n_vacancies.extend(new_top_n_vac)
    
    return top_n_vacancies

def get_predictions_surprise(user_id, model, vac_set, n_predict = 5):
    # Создадим тест датасет для данного юзера:
    test_set = [[user_id, item, 4.] for item in vac_set]        
    
    predictions = model.test(test_set)
    top_n = sorted(predictions, key=lambda x: x.est, reverse=True)[:n_predict]
    return [pred.iid for pred in top_n]

def get_predictions_implicit(user_id, model, filtered_set, impl_set, n_predict = 5):
    user_item = impl_set[0]
    id_cat_code = impl_set[1][user_id]
    vac_cat_codes = impl_set[2]
    cat_vac_codes = impl_set[3]
    filter_items = np.array([vac_cat_codes[i] for i in filtered_set])
    ids, scores = model.recommend(id_cat_code,
                                  user_item[id_cat_code],
                                  N=n_predict,
                                  filter_items=filter_items,
                                  filter_already_liked_items=True)
    return [cat_vac_codes[i] for i in ids]    

def calc_correct_recomend_number(predict_col, vac_col):
    """Функция для определения количества совпадающих вакансий в двух списках"""
    return len(set(predict_col) & set(vac_col))


def common_for_cookie(cookie_id, n_predict = 5):
    """Функция для определения common_id от cookie_id для обучения рекомендательной системы"""
    common_freq = train_df[train_df['cookie_id'] == cookie_id][id_column_name].value_counts(normalize=True)
    common_ids = {com_id: round(freq*n_predict) for com_id, freq in zip(common_freq.index.to_list(), common_freq.to_list())}
    return common_ids


def get_prec_n(res_df, num_corr_column, n_predict = 5):
    """Функция для расчета precision@n метрики"""
    corr_recommend = res_df[num_corr_column].sum()
    prec_5 = corr_recommend / (n_predict * res_df.shape[0])
    return {'num_corr_recomend': corr_recommend, f'precision@{n_predict}': prec_5}

In [46]:
# Копируем датасет чтобы не менять изначальный

test_df_with_pred = test_6.copy()

In [ ]:
# Задаем словари с моделями и датасетами чтобы перебрать их все

models_dict = {
    'SVD_20ep_rat0-1': [svd_20epochs_0_1, 'surprise'],
}

set_dict = {
    'top80': top80vac,
    'top95_10': top95vac_from10
}

In [47]:
# Импортируем tqdm для progress bar
from tqdm.auto import tqdm

def get_res_df(models_dict, set_dict, n_predict = 5):
    res_dict = {}
    for model_name in tqdm(models_dict):        
        for set_name in tqdm(set_dict):        
            # Создаем столбец с предсказаниями
            predict_col_name = f'predict_id_{model_name}_{set_name}_n_{n_pred}'            
            test_df_with_pred[predict_col_name] = test_df_with_pred['cookie_id'].apply(get_predictions,
                                                                                       model_info = models_dict[model_name],
                                                                                       initial_set = set_dict[set_name],
                                                                                       n_predict = n_pred)                  
            
            # Создаем столбец с количеством правильных предсказаний для каждого id
            num_corr_col_name = f'num_corr_{model_name}_{set_name}_n_{n_pred}'        
            test_df_with_pred[num_corr_col_name] = test_df_with_pred.apply(
                lambda x: calc_correct_recomend_number(x[predict_col_name], x['vacancy_id_']), axis=1
            )
            
            # Считаем precision@n метрику
            res_dict[model_name+'_'+set_name] = get_prec_n(test_df_with_pred,
                                                           num_corr_column = num_corr_col_name,
                                                           n_predict = n_pred)
    return pd.DataFrame.from_dict(res_dict, orient='index')

In [ ]:
n_pred = 5

res_df = get_res_df(models_dict, set_dict, n_predict = n_pred)
res_df

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

,num_corr_recomend,precision@5
SVD_20ep_rat0-1_top80,26,0.006736
SVD_20ep_rat0-1_top95_10,91,0.023575


In [ ]:
# Создание списка неактульных вакансий, с которомы давно не было взаимодействий
# 78 973 вакансии не были провзаимодействованы за последние 2 недели рассматриваемого периода (prec = 0.031)
# 99 046 вакансии не были провзаимодействованы за последнюю неделю (prec = 0.033)
# 117 944 вакансии не были провзаимодействованы за последние 3 дня (prec = 0.036)
# 127 405 вакансии не были провзаимодействованы за последние 2 дня (prec = 0.038)
# 140 242 вакансии не были провзаимодействованы за последний 1 день - 0.04
non_valid_vacs = []
for vac, group in train_df.groupby('vacancy_id_'):
    max_date = group['event_date'].max()
    if max_date <= '2022-09-30':
        non_valid_vacs.append(vac)
len(non_valid_vacs)

140242

In [ ]:
valid_vacs = train_matrix[~train_matrix['vacancy_id_'].isin(non_valid_vacs)]['vacancy_id_'].unique()
len(valid_vacs)

19925

In [48]:
# Задаем словари с моделями и датасетами чтобы перебрать их все

models_dict = {
    'Impl_ALS_rat0-1': [Impl_ALS_0_1, 'implicit', Impl_0_1_set]
}

set_dict = {
    #'top200': top200vac,
    'top1000': top1000vac
   # 'non_val_vacs': valid_vacs
}

In [49]:
n_pred = 5

res_df = get_res_df(models_dict, set_dict, n_predict = n_pred)
res_df

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

KeyError: ignored

In [ ]:
pd.set_option('display.max_rows', None)
res_df.sort_values(by=['precision@5'], ascending=False)

,num_corr_recomend,precision@5
Impl_ALS_rat0-1_non_val_vacs,121,0.031347
Impl_ALS_rat0-1_top1000,88,0.022798
Impl_ALS_rat0-1_top200,60,0.015544
